In [26]:
import yfinance as yf
import plotly.express as px

In [27]:
df = yf.download("TCS.NS", period="6mo", multi_level_index=False)

[*********************100%***********************]  1 of 1 completed


In [ ]:
# Calculating the 5-day and 20-day Simple Moving Averages (SMA)
df["5_Day_SMA"] = df["Close"].rolling(window=5).mean()
df["20_Day_SMA"] = df["Close"].rolling(window=20).mean()

# Determinig the trend
# 1 if 5-day SMA > 20-day SMA, -1 if 5-day SMA < 20-day SMA
df["Trend"] = 0
df.loc[df["5_Day_SMA"] > df["20_Day_SMA"], "Trend"] = 1  # 1 indicates a recent uptrend
df.loc[df["5_Day_SMA"] < df["20_Day_SMA"], "Trend"] = -1  # -1 indicates a recent downtrend

# Calculating the change in trend to identify crossover points
df["Shift"] = df["Trend"].diff()

# Buy signal: when trend changes from -1 to 1 (bullish crossover)
df["Buy_Signals"] = df["Shift"] == 2
# Sell signal: when trend changes from 1 to -1 (bearish crossover)
df["Sell_Signals"] = df["Shift"] == -2

In [ ]:
# Plotting the Close price, 5-day SMA, and 20-day SMA
fig = px.line(
    df,
    x=df.index,
    y=["Close", "5_Day_SMA", "20_Day_SMA"],
    labels={"variable": "Legend", "value": "Price","5_Day_SMA": "5 Day SMA"},
    title="TCS - SMA Crossover Strategy",
    height=600,
    width=1200
)

# Adding green triangle markers for buy signals
fig.add_scatter(
    x=df[df["Buy_Signals"]].index,
    y=df.loc[df["Buy_Signals"], "5_Day_SMA"],
    mode="markers",
    marker=dict(symbol="triangle-up", size=20, color="green"),
    name="Buy",
)

# Adding red triangle markers for sell signals
fig.add_scatter(
    x=df[df["Sell_Signals"]].index,
    y=df.loc[df["Sell_Signals"], "20_Day_SMA"],
    mode="markers",
    marker=dict(symbol="triangle-down", size=20, color="red"),
    name="Sell",
)
fig